In [1]:
import cvxpy as cp
import numpy as np
from scipy.linalg import sqrtm
import random as r
import pandas as pd
from qpth.qp import QPFunction
import torch
from sklearn.datasets import make_spd_matrix

In [2]:
global n
n=6       #length of the optimiation variable
global m
m=6       #no.of inequality constraints
global p
p=3       #no.of equality constraints

In [3]:
def find_valid(seeds):
    #find seeds for which the corresponding randomly generated problem yields a solution (from cvxpy)
    seeds_old=seeds.copy()
    seeds_old_old=seeds_old.copy()
    while True:        
        while True: 
            for k,s in enumerate(seeds):
                r.seed(s)
                P=make_spd_matrix(n, random_state=s)
                P = sqrtm(P)
                q=np.array([r.randint(0,9) for i in range(n)]).reshape(n).astype(float)
                G=np.array([r.randint(0,9)*((-1)**r.randint(0,1)) for i in range(m*n)]).reshape(m,n).astype(float)
                h=np.array([0 for i in range(m)]).reshape(m).astype(float)
                A=np.array([r.randint(0,9) for i in range(p*n)]).reshape(p,n).astype(float)
                b=np.array([r.randint(0,9) for i in range(p)]).reshape(p).astype(float)
                try:
                    x,y=qp_cvxpy(n,P,q,G,h,A,b)
                except:
                    #if solving the problem creates error, remove this seed from list
                    seeds.remove(s)
                    continue
                #if the problem is infeasible, y is None; if so remove this seed from list
                if(np.array(y).any()==None):
                    seeds.remove(s)
            if len(seeds_old)==len(seeds):
                break
            else:
                seeds_old=seeds
        if len(seeds_old_old)==len(seeds_old):
            break
        seeds_old_old=seeds_old
    print("No.of problems generated: ",len(seeds)) 
    return(seeds)

In [4]:
def qp_cvxpy(n,P,q,G,h,A,b):
    x= cp.Variable(n)
    objective=cp.Minimize((1/2)*cp.sum_squares(P*x) + q.T @ x)
    constraints=[G@x <=h,A@x ==b]
    prob=cp.Problem(objective,constraints)
    prob.solve(verbose=False)
    cvxpy_optimum= prob.value
    cvxpy_solution=x.value
    return cvxpy_optimum,cvxpy_solution

In [5]:
#initialize seeds for generating random quadratic problems
try_seeds=[i for i in range(100,100000,10)]
# print(len(try_seeds))
seeds= find_valid(try_seeds)

No.of problems generated:  6417


In [6]:
columns=["seed"]+[str(i)+"cp" for i in range(n)]
results=pd.DataFrame([],columns=columns)
for k,s in enumerate(seeds):
    r.seed(s)
    P=make_spd_matrix(n, random_state=s)
    q=np.array([r.randint(0,9) for i in range(n)]).reshape(n).astype(float)
    G=np.array([r.randint(0,9)*((-1)**r.randint(0,1)) for i in range(m*n)]).reshape(m,n).astype(float)
    h=np.array([0 for i in range(m)]).reshape(n).astype(float)
    A=np.array([r.randint(0,9) for i in range(p*n)]).reshape(p,n).astype(float)
    b=np.array([r.randint(0,9) for i in range(p)]).reshape(p).astype(float)
    
    P_tch=torch.from_numpy(P)
    q_tch=torch.from_numpy(q)
    G_tch=torch.from_numpy(G)
    h_tch=torch.from_numpy(h)
    A_tch=torch.from_numpy(A)
    b_tch=torch.from_numpy(b)
    
    
    P = sqrtm(P)
    x,y=qp_cvxpy(n,P,q,G,h,A,b)
    re=pd.Series({"seed":s})
    for i in range(n):
        re=re.append(pd.Series({str(i)+"cp":np.round(y[i],6)}))
    
    qpf = QPFunction()
    try:
        solution = qpf(P_tch, q_tch, G_tch, h_tch, A_tch, b_tch)
        qpth_solution=solution.detach().numpy()[0]
    except:
        qpth_solution=[0]*n
    for i in range(n):
        re=re.append(pd.Series({str(i)+"qpt":np.round(qpth_solution[i],6)}))
    results=results.append(re,ignore_index=True )
    


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i

...


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i

In [7]:
error=np.zeros(len(results))
for i in range(n):
    error+=results[str(i)+"cp"]-results[str(i)+"qpt"]
results["cp-qpt-error"]=error    
results["cp-qpt-er_flag"]=error!=0

print("No of solved problems",len(seeds))
print("No of inaccurate results",len(error[error!=0]))

results[results["cp-qpt-er_flag"]]

No of solved problems 6417
No of inaccurate results 313


,seed,0cp,1cp,2cp,3cp,4cp,5cp,0qpt,1qpt,2qpt,3qpt,4qpt,5qpt,cp-qpt-error,cp-qpt-er_flag
15,360.0,-0.400090,-8.847799,4.028417,-5.439977,1.761969,2.236549,0.464182,-1.552062,1.138961,-0.798860,0.279653,0.074129,-6.266934,True
18,400.0,5.238220,-1.923429,5.077225,5.727094,-6.282723,6.267016,1.262468,1.062144,-0.506128,2.006004,-1.288326,0.155147,11.412094,True
72,1340.0,-1.221996,3.452352,-1.260098,-4.570263,-1.121232,2.466149,0.193232,0.979446,-0.436470,-1.311860,-0.307930,0.882552,-2.254058,True
174,2920.0,-9.338583,-9.881890,5.220472,5.653543,6.244094,10.157480,-7.439791,-7.530245,4.875014,3.822770,4.354812,8.188102,1.784454,True
183,3070.0,-4.483220,2.040414,7.561966,8.055154,-8.131626,-3.116473,0.555620,-0.247828,0.545141,0.664442,-0.273798,-0.101710,0.784348,True
190,3150.0,-18.274590,-27.152459,8.861475,9.636066,7.383607,7.514754,0.315030,-0.230616,0.209725,0.116471,0.004432,0.509967,-12.956156,True
203,3330.0,-17.249111,-6.010441,7.250754,-11.677336,1.789343,17.291923,-1.343228,0.638510,-0.302427,0.013072,0.574842,0.100492,-8.286129,True
221,3650.0,-2.358418,-2.869544,-4.816039,-1.072076,7.109924,-3.063688,-2.352926,-2.883133,-4.815594,-1.070506,7.103760,-3.057295,0.005853,True
285,4590.0,3.362302,-1.489500,-2.054629,-1.207035,2.713798,-0.921727,0.488899,0.856958,-0.518210,-0.239803,0.053463,0.208193,-0.446291,True
318,5130.0,-20.239846,19.914573,-5.128942,22.540444,-9.099782,5.446464,-0.336779,-0.241715,1.080021,0.297595,-0.320653,0.776832,12.177610,True


In [8]:
results.to_csv("test_cases.csv")

In [ ]:
results.head()